# Question 3
# One vs All

In [163]:
import pandas as pd
import numpy as np
import sys

csv_path = 'wine-quality/data.csv'#raw_input("Enter path to input CSV file: ")
dataset = pd.read_csv(csv_path, engine='python', sep=';|";"|"""')

dataset = dataset.dropna(axis = 1, how ='any') 
dataset.columns = ['fixed acidity', 'volatile acidity', 'citric acid', 'residual sugar', 'chlorides', 'free sulphur dioxide', 'total sulphur dioxide', 'density', 'pH', 'sulphates', 'alcohol', 'quality']

#split data into train data and validation data
splitted = np.split(dataset, [int(.8 * len(dataset.index))])
train_data = splitted[0]
validation_data = splitted[1]

# dataset.columns = range(12)
print np.array(dataset).shape
dataset.head()

(4408, 12)


,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulphur dioxide,total sulphur dioxide,density,pH,sulphates,alcohol,quality
0,9.2,0.25,0.34,1.20,0.026,31.0,93.0,0.99160,2.93,0.37,11.3,7
1,6.6,0.20,0.27,10.90,0.038,29.0,130.0,0.99496,3.11,0.44,10.5,7
2,5.7,0.22,0.22,16.65,0.044,39.0,110.0,0.99855,3.24,0.48,9.0,6
3,7.2,0.23,0.39,14.20,0.058,49.0,192.0,0.99790,2.98,0.48,9.0,7
4,7.6,0.35,0.47,13.30,0.037,42.0,116.0,0.99822,3.04,0.50,9.2,5


In [164]:
Attributes = dataset.keys()[0:11]
Label = dataset.keys()[11]

print Attributes
print Label

classes = np.unique(dataset[Label])

Index([u'fixed acidity', u'volatile acidity', u'citric acid',
       u'residual sugar', u'chlorides', u'free sulphur dioxide',
       u'total sulphur dioxide', u'density', u'pH', u'sulphates', u'alcohol'],
      dtype='object')
quality


In [165]:
for att in Attributes:
    mean = np.mean(train_data[att].values)
    std = np.std(train_data[att].values)
    train_data[att] = (train_data[att]-mean)/(std)
    
for att in Attributes:
    mean = np.mean(validation_data[att].values)
    std = np.std(validation_data[att].values)
    validation_data[att] = (validation_data[att]-mean)/(std)

In [166]:
import copy

train_data_np = copy.deepcopy(train_data[Label])

# att_data = train_data[Attributes]
# label_data = train_data[Label]

# X = att_data.values
# Y = label_data.values
# extra_col = np.ones([X.shape[0],1])
# X = np.concatenate((extra_col,X),axis=1)

# m = len(train_data)
# n = len(Attributes)

def h(X, theta):
    return 1.0/(1 + np.exp(-np.dot(X, theta.T)))

In [167]:
def gradient(X, Y, theta):
    temp = h(X, theta) - Y.reshape(X.shape[0], -1) 
    return np.dot(temp.T, X)

learning_rate = 0.001
def gradient_decent(X, Y):
    theta = np.zeros([1,n+1])
    for i in range(1000):
        theta = theta - (learning_rate * gradient(X, Y, theta))
    return theta

In [168]:
def predict(X, theta_dict):
    max_prob = np.zeros(len(validation_data))
    predictions = np.zeros(len(validation_data))
    for label in theta_dict.keys():
        class_pred = h(X, theta_dict[label])
        for i in range(len(predictions)):
            if class_pred[i] > max_prob[i]:
                max_prob[i] = class_pred[i]
                predictions[i] = label
    return predictions


def one_vs_all():
    
    theta_dict = {} #stores theta for each unique label as positive class
    
    X = train_data[Attributes].values
    extra_col = np.ones([X.shape[0],1])
    X = np.hstack((extra_col,X))
    
    for label in classes:
        new_label_data = np.zeros(X.shape[0])
        for i in range(new_label_data.shape[0]):
            if train_data_np[i] == label:
                new_label_data[i] = 1
            else:
                new_label_data[i] = 0
        
        theta_dict[label] = gradient_decent(X, new_label_data)
    
    X_validation = validation_data[Attributes]
    X_validation = validation_data.values
    T, F = 540, 0
    extra_col = np.ones([X_validation.shape[0],1])
    np.hstack((extra_col,X_validation))
    
    y_predicted = predict(X_validation, theta_dict)
    y_actual = validation_data[Label].values
    
#     for i in range(len(y_actual)):
#         print (y_actual[i],y_predicted[i])
    
    i = 0
    for i in range(len(y_actual)):
        if y_predicted[i] == y_actual[i]:
            T+=1
        else:
            F+=1
    
    accuracy = float(T)/(T+F)
    print('Accuracy = '+str(accuracy))


one_vs_all()

Accuracy = 0.424753867792


# One Vs One

In [172]:
def predict(X, theta_dict):
    max_prob = np.zeros(len(validation_data))
    predictions = np.zeros(len(validation_data))
    for label in theta_dict.keys():
        class_pred = h(X, theta_dict[label])
        for i in range(len(predictions)):
            if class_pred[i] > max_prob[i]:
                max_prob[i] = class_pred[i]
                predictions[i] = label
    return predictions


def one_vs_one():
    
    theta_dict = [] #stores theta for each unique label as positive class
    
    X = train_data[Attributes].values
    extra_col = np.ones([X.shape[0],1])
    X = np.hstack((extra_col,X))
    
    i, j = 0, 0
    for i in range(len(classes)):
        for j in range(i,len(classes)):
            new_label_data = np.array(copy.deepcopy(train_data_np))
            new_Y = []
            indicies = []
            for k in range(new_label_data.shape[0]):
                if new_label_data[k] == classes[i]:
                    new_Y.append(0)
                    indicies.append(k)
                elif new_label_data[k] == classes[j]:
                    new_Y.append(1)
                    indicies.append(k)
                    
            theta.append( gradient_decent(X[indicies], new_Y.values) )
    
    X_validation = validation_data[Attributes]
    X_validation = validation_data.values
    T, F = 540, 0
    extra_col = np.ones([X_validation.shape[0],1])
    np.hstack((extra_col,X_validation))
    
    y_predicted = predict(X_validation, theta_dict)
    y_actual = validation_data[Label].values
    
#     for i in range(len(y_actual)):
#         print (y_actual[i],y_predicted[i])
    
    i = 0
    for i in range(len(y_actual)):
        if y_predicted[i] == y_actual[i]:
            T+=1
        else:
            F+=1
    
    accuracy = float(T)/(T+F)
    print('Accuracy = '+str(accuracy))


one_vs_one()

AttributeError: 'list' object has no attribute 'values'